# Feature Selection for Lending Club Data
## COMP647 Assignment 03
### Student ID: 1163127

This notebook implements feature selection techniques including:
- Univariate statistical tests (ANOVA, Chi-Square)
- Wrapper methods (Recursive Feature Elimination)
- Embedded methods (Random Forest importance)

Based on LAB5 materials and course teachings.

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Feature selection libraries
from sklearn.feature_selection import (
    SelectKBest, f_classif, chi2, 
    RFE, RFECV,
    SelectFromModel
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print("Libraries imported successfully")

In [ ]:
# Load preprocessed data from previous notebooks
try:
    # Load the processed sample data
    df = pd.read_csv('../data/processed/accepted_sample_10000.csv')
    print(f"Data loaded successfully: {df.shape}")
    print(f"Columns: {len(df.columns)}")
    
    # Display basic info
    print("\nDataset Preview:")
    print(df.head())
    
    # Identify target variable
    target_column = 'loan_status' if 'loan_status' in df.columns else None
    if target_column:
        print(f"\nTarget variable: {target_column}")
        print(f"Target distribution:")
        print(df[target_column].value_counts())
    else:
        print("\nWarning: loan_status column not found")
        
except FileNotFoundError:
    print("Processed data not found. Please run Assignment 02 notebooks first.")
    df = pd.DataFrame()
    target_column = None

## Univariate Statistical Tests
### ANOVA F-test for Numerical Features
Based on LAB5 materials - measures linear dependency between features and target

In [ ]:
# ANOVA F-test Implementation
from sklearn.feature_selection import SelectKBest, f_classif

def apply_anova_ftest(X, y, k=10):
    """
    Apply ANOVA F-test for feature selection
    
    Parameters:
    X: DataFrame with numerical features
    y: target variable (classification)
    k: number of top features to select
    
    Returns:
    selected_features, results_df, selector
    """
    print("Applying ANOVA F-test for numerical features...")
    
    selector = SelectKBest(score_func=f_classif, k=min(k, X.shape[1]))
    selector.fit(X, y)
    
    # Get results
    results_df = pd.DataFrame({
        'feature': X.columns,
        'f_score': selector.scores_,
        'selected': selector.get_support()
    }).sort_values('f_score', ascending=False)
    
    selected_features = results_df[results_df['selected']]['feature'].tolist()
    
    print(f"Selected {len(selected_features)} features")
    print("\nTop 5 features:")
    for idx, row in results_df.head(5).iterrows():
        print(f"  {row['feature']}: F-score = {row['f_score']:.2f}")
    
    return selected_features, results_df, selector

# Apply ANOVA if data is available
if not df.empty and target_column:
    # Get numerical features
    num_features = df.select_dtypes(include=[np.number]).columns.tolist()
    num_features = [c for c in num_features if c != target_column and not c.endswith('_id')]
    
    if len(num_features) >= 5:
        X_num = df[num_features].fillna(df[num_features].mean())
        
        # Prepare target
        le = LabelEncoder()
        y = le.fit_transform(df[target_column].fillna('Unknown'))
        
        # Apply ANOVA
        sel_anova, anova_results, anova_sel = apply_anova_ftest(X_num, y, k=10)
        
        # Visualize
        plt.figure(figsize=(10, 6))
        top15 = anova_results.head(15)
        plt.bar(range(len(top15)), top15['f_score'])
        plt.xticks(range(len(top15)), top15['feature'], rotation=45, ha='right')
        plt.title('Top 15 Features - ANOVA F-test')
        plt.ylabel('F-Score')
        plt.tight_layout()
        plt.show()
    else:
        print("Not enough numerical features for ANOVA")
        sel_anova = []
else:
    print("No data for ANOVA demo")
    sel_anova = []

### Chi-Square Test for Categorical Features
Based on LAB5 materials - measures dependency between categorical features and target

In [ ]:
# Chi-Square Test Implementation
from sklearn.feature_selection import chi2

def apply_chi_square(X, y, k=5):
    """Apply Chi-Square test for categorical features"""
    print("Applying Chi-Square test for categorical features...")
    
    selector = SelectKBest(score_func=chi2, k=min(k, X.shape[1]))
    selector.fit(X, y)
    
    results_df = pd.DataFrame({
        'feature': X.columns,
        'chi2_score': selector.scores_,
        'selected': selector.get_support()
    }).sort_values('chi2_score', ascending=False)
    
    selected_features = results_df[results_df['selected']]['feature'].tolist()
    
    print(f"Selected {len(selected_features)} features")
    print("\nTop features:")
    for idx, row in results_df.head(5).iterrows():
        print(f"  {row['feature']}: Chi2 = {row['chi2_score']:.2f}")
    
    return selected_features, results_df, selector

# Apply Chi-Square if data available
if not df.empty and target_column:
    cat_features = df.select_dtypes(include=['object']).columns.tolist()
    if target_column in cat_features:
        cat_features.remove(target_column)
    
    if len(cat_features) >= 3:
        # Encode categorical features
        X_cat = df[cat_features[:10]].copy()
        for col in X_cat.columns:
            le = LabelEncoder()
            X_cat[col] = le.fit_transform(X_cat[col].fillna('Unknown'))
        
        le = LabelEncoder()
        y = le.fit_transform(df[target_column].fillna('Unknown'))
        
        sel_chi, chi_results, chi_sel = apply_chi_square(X_cat, y, k=5)
        
        # Visualize
        plt.figure(figsize=(10, 6))
        top10 = chi_results.head(10)
        plt.bar(range(len(top10)), top10['chi2_score'])
        plt.xticks(range(len(top10)), top10['feature'], rotation=45, ha='right')
        plt.title('Top 10 Features - Chi-Square Test')
        plt.ylabel('Chi-Square Score')
        plt.tight_layout()
        plt.show()
    else:
        print("Not enough categorical features")
        sel_chi = []
else:
    sel_chi = []

## Wrapper Methods
### Recursive Feature Elimination (RFE)
Backward elimination using model weights

In [ ]:
# RFE Implementation
from sklearn.feature_selection import RFE

def apply_rfe(X, y, n_features=10):
    """Apply Recursive Feature Elimination"""
    print("Applying RFE with Logistic Regression...")
    
    estimator = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
    selector = RFE(estimator, n_features_to_select=min(n_features, X.shape[1]), step=1)
    selector.fit(X, y)
    
    rankings = pd.DataFrame({
        'feature': X.columns,
        'ranking': selector.ranking_,
        'selected': selector.get_support()
    }).sort_values('ranking')
    
    selected = rankings[rankings['selected']]['feature'].tolist()
    
    print(f"Selected {len(selected)} features")
    print("\nSelected features:")
    for f in selected[:5]:
        print(f"  {f}")
    
    return selected, rankings, selector

# Apply RFE
if not df.empty and target_column:
    num_features = df.select_dtypes(include=[np.number]).columns.tolist()
    num_features = [c for c in num_features if c != target_column and not c.endswith('_id')]
    
    if len(num_features) >= 5:
        X_rfe = df[num_features[:15]].fillna(df[num_features[:15]].mean())
        scaler = StandardScaler()
        X_rfe = pd.DataFrame(scaler.fit_transform(X_rfe), columns=X_rfe.columns)
        
        le = LabelEncoder()
        y = le.fit_transform(df[target_column].fillna('Unknown'))
        if len(np.unique(y)) > 2:
            y = (y == y[0]).astype(int)
        
        sel_rfe, rfe_rank, rfe_sel = apply_rfe(X_rfe, y, n_features=8)
        
        # Visualize rankings
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(rfe_rank)), rfe_rank['ranking'])
        plt.xticks(range(len(rfe_rank)), rfe_rank['feature'], rotation=45, ha='right')
        plt.title('Feature Rankings - RFE (1=best)')
        plt.ylabel('Ranking')
        plt.tight_layout()
        plt.show()
    else:
        sel_rfe = []
else:
    sel_rfe = []

## Embedded Methods
### Random Forest Feature Importance
Model-based selection using impurity decrease

In [ ]:
# Random Forest Feature Importance
from sklearn.ensemble import RandomForestClassifier

def apply_rf_importance(X, y, n_features=10):
    """Apply Random Forest feature selection"""
    print("Applying Random Forest feature importance...")
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    rf.fit(X, y)
    
    importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    selected = importance_df.head(n_features)['feature'].tolist()
    
    print(f"Selected {len(selected)} features")
    print("\nTop features:")
    for idx, row in importance_df.head(5).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")
    
    return selected, importance_df, rf

# Apply Random Forest
if not df.empty and target_column:
    num_features = df.select_dtypes(include=[np.number]).columns.tolist()
    num_features = [c for c in num_features if c != target_column and not c.endswith('_id')]
    
    if len(num_features) >= 5:
        X_rf = df[num_features[:15]].fillna(df[num_features[:15]].mean())
        
        le = LabelEncoder()
        y = le.fit_transform(df[target_column].fillna('Unknown'))
        if len(np.unique(y)) > 2:
            y = (y == y[0]).astype(int)
        
        sel_rf, rf_imp, rf_model = apply_rf_importance(X_rf, y, n_features=10)
        
        # Visualize
        plt.figure(figsize=(10, 6))
        top15 = rf_imp.head(15)
        plt.barh(range(len(top15)), top15['importance'])
        plt.yticks(range(len(top15)), top15['feature'])
        plt.title('Top 15 Features - Random Forest Importance')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
    else:
        sel_rf = []
else:
    sel_rf = []

## Comparison of Methods
Compare results from different feature selection approaches

In [ ]:
# Compare all methods
print("=== Feature Selection Methods Comparison ===\n")

methods_summary = []

if sel_anova:
    methods_summary.append({'Method': 'ANOVA F-test', 
                           'Count': len(sel_anova),
                           'Top_3': ', '.join(sel_anova[:3])})

if sel_chi:
    methods_summary.append({'Method': 'Chi-Square', 
                           'Count': len(sel_chi),
                           'Top_3': ', '.join(sel_chi[:3])})

if sel_rfe:
    methods_summary.append({'Method': 'RFE', 
                           'Count': len(sel_rfe),
                           'Top_3': ', '.join(sel_rfe[:3])})

if sel_rf:
    methods_summary.append({'Method': 'Random Forest', 
                           'Count': len(sel_rf),
                           'Top_3': ', '.join(sel_rf[:3])})

if methods_summary:
    summary_df = pd.DataFrame(methods_summary)
    print(summary_df.to_string(index=False))
    
    print("\n=== Recommendations ===")
    print("ANOVA F-test: Best for linear relationships (numerical features)")
    print("Chi-Square: Best for categorical features")
    print("RFE: Considers feature interactions")
    print("Random Forest: Captures non-linear patterns")
    print("\nBest practice: Combine multiple methods for robust selection")
else:
    print("No methods completed successfully")